# Comparing UKBB and BioVU genome wide hits

-manhattan plots   
-any shared snps


In [1]:
import os, sys
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.colors
from datetime import datetime

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
from IPython.core.display import display, HTML    
display(HTML("<style>.container {width:90% !important; }</style>"))
%matplotlib inline 
np.set_printoptions(precision=5, suppress=True) 

DATE = datetime.now().strftime('%Y-%m-%d')

In [2]:
import matplotlib.font_manager as fm
fpath='/dors/capra_lab/users/abraha1/conda/envs/py36_r_ml/lib/python3.6/site-packages/matplotlib/mpl-data/fonts/ttf/Arial.ttf'
prop = fm.FontProperties(fname=fpath, size=16)
bigprop = fm.FontProperties(fname=fpath, size=20)

In [3]:
sys.path.append("/dors/capra_lab/users/abraha1/bin/python_modules/assocplots")
from assocplots.manhattan import *

In [4]:
import rpy2.rinterface 
%load_ext rpy2.ipython

In [34]:
GWAS_P_THRESH = 5*10**-8
SUGG_GWAS_P_THRESH = 1*10**-6
MISS_P_THRESHOLD = 0.00001

In [6]:
# PATHS
root="/dors/capra_lab/users/abraha1/prelim_studies/katja_biobank/"
ukbb_path = os.path.join(root, "data/assoc_from_katja_2019_08_20/20190819_gwas_pca12_centers_age_FINAL.csv")
biovu_path= os.path.join(root, "results/2019_07_21_logistic/2019_07_21_logistic.assoc.logistic")

imp_bv_snps_path = "/dors/capra_lab/users/abraha1/prelim_studies/katja_biobank/data/snps_imputed_20190827_EU_filt1.txt"

ld_snps_ukbb="/dors/capra_lab/users/abraha1/prelim_studies/katja_biobank/results/manuscript/ukbb_ld_w_sig_hits_super_euro_1kg.txt"
ld_snps_biovu="/dors/capra_lab/users/abraha1/prelim_studies/katja_biobank/results/manuscript/biovu_ld_w_sig_hits_super_euro_1kg.txt"

output_dir="/dors/capra_lab/users/abraha1/prelim_studies/katja_biobank/results/manuscript/manhattan_plots"

# load and filter

In [7]:
# LOAD 
raw_uk_df = pd.read_csv( ukbb_path, sep=",")
raw_bv_df = pd.read_csv( biovu_path, sep="\s+")
imp_snps_df = pd.read_csv(imp_bv_snps_path, header=None, names=['snps'])

In [8]:
uk_df = raw_uk_df.loc[raw_uk_df.missing_p > MISS_P_THRESHOLD].copy() 
print("Removed {:,} out of {:,} snps with singificant missingness b/w cases and controls.".format(raw_uk_df.shape[0] - uk_df.shape[0], raw_uk_df.shape[0]))
uk_df['chr_pos'] = uk_df.CHR.map(str) + ":" + uk_df.BP.map(str)

Removed 28,714 out of 648,754 snps with singificant missingness b/w cases and controls.


In [9]:
bv_df = raw_bv_df.loc[(raw_bv_df.TEST == "ADD") & (raw_bv_df.CHR < 23),].copy()
bv_df['chr_pos'] = bv_df.CHR.map(str) + ":" + bv_df.BP.map(str)

# significant regions

In [10]:
sig_uk_df = uk_df.loc[uk_df.P < GWAS_P_THRESH].copy()
sig_bv_df = bv_df.loc[(bv_df.P < GWAS_P_THRESH)].copy()

In [11]:
print("UKBB: Number of genome wide significant {:,}".format(sig_uk_df.shape[0]))
print("BV: Number of genome wide significant {:,}".format(sig_bv_df.shape[0]))

UKBB: Number of genome wide significant 8
BV: Number of genome wide significant 5


In [12]:
# there is no overlap of the significant variants tested (by position) between the two datsets 
bv_df[bv_df.chr_pos.isin(sig_uk_df.chr_pos)]
uk_df[uk_df.chr_pos.isin(sig_bv_df.chr_pos)]

,CHR,SNP,BP,A1,TEST,NMISS,OR,STAT,P,chr_pos


,CHR,SNP,BP,A1,TEST,NMISS,OR,STAT,P,missing_p,chr_pos


In [15]:
rename_bv_snps = {'JHU_3.16652239' :'rs9870157',
'JHU_13.20119335' :'rs9508454',
'14:35761675-C-G' :'rs1048990'}

In [19]:
rename_bv_snps.keys()

dict_keys(['JHU_3.16652239', 'JHU_13.20119335', '14:35761675-C-G'])

In [25]:
# rename snp IDs in biovu to 
bv_df.SNP = bv_df.SNP.apply(lambda x: rename_bv_snps[x] if x in rename_bv_snps.keys() else x)
sig_bv_df.SNP = sig_bv_df.SNP.apply(lambda x: rename_bv_snps[x] if x in rename_bv_snps.keys() else x)


## suggestive significance

In [36]:
sug_sig_uk_df = uk_df.loc[uk_df.P < SUGG_GWAS_P_THRESH].copy()
sug_sig_bv_df = bv_df.loc[(bv_df.P < SUGG_GWAS_P_THRESH)].copy()

In [37]:
print("UKBB: Number of suggestive genome wide significant {:,}".format(sug_sig_uk_df.shape[0]))
print("BV: Number of suggestive genome wide significant {:,}".format(sug_sig_bv_df.shape[0]))

UKBB: Number of suggestive genome wide significant 14
BV: Number of suggestive genome wide significant 8


# one manhattan plot

In [28]:
%%R 
# set plot parameters
theme_Publication <- function(base_size=14, base_family="Arial") {
      library(grid)
      library(ggthemes)
      (theme_foundation(base_size=base_size, base_family=base_family)
       + theme(plot.title = element_text(face = "plain", size = rel(1.2), hjust = 0.5),
               text = element_text(),
               panel.background = element_rect(colour = NA),
               plot.background = element_rect(colour = NA),
               panel.border = element_rect(colour = NA),
               axis.title = element_text(face = "plain",size = rel(1)),
               axis.title.y = element_text(angle=90,vjust =2),
               axis.title.x = element_text(vjust = -0.2),
               axis.text = element_text(), 
               axis.line = element_line(colour="black"),
               axis.ticks = element_line(),
               panel.grid.major = element_line(colour="#f0f0f0"),
               panel.grid.major.y = element_blank(),
               panel.grid.minor = element_blank(),
               legend.key = element_rect(colour = NA),
               legend.position = "bottom",
               legend.direction = "horizontal",
               legend.key.size= unit(0.2, "cm"),
               legend.margin = unit(0, "cm"),
               legend.title = element_text(face="italic"),
               plot.margin=unit(c(10,5,5,5),"mm"),
               strip.background=element_rect(colour="#f0f0f0",fill="#f0f0f0"),
               strip.text = element_text(face="plain")
          ))
      
}

scale_fill_Publication <- function(...){
      library(scales)
      discrete_scale("fill","Publication",manual_pal(values = c("#386cb0","#fdb462","#7fc97f","#ef3b2c","#662506","#a6cee3","#fb9a99","#984ea3","#ffff33")), ...)

}

scale_colour_Publication <- function(...){
      library(scales)
      discrete_scale("colour","Publication",manual_pal(values = c("#386cb0","#fdb462","#7fc97f","#ef3b2c","#662506","#a6cee3","#fb9a99","#984ea3","#ffff33")), ...)

}

In [29]:
%%R 
library(ggplot2)
library(readr)
library(ggrepel)
library(ggplot2)
library(dplyr)
library(RColorBrewer)

gg.manhattan <- function(df, threshold, hlight, col, ylims, title){
    sig = 5e-8 # significant threshold line
    sugg = 1e-6 # suggestive threshold line
    
    df.tmp = df %>% group_by(CHR) %>% summarise(chr_len=max(BP)) %>% mutate(tot=cumsum(chr_len)-chr_len) %>% select(-chr_len) %>% left_join(df, ., by=c("CHR"="CHR"))
    df.tmp = df.tmp %>% arrange(CHR, BP) %>% mutate(BPcum=BP+tot) %>% mutate( is_highlight=ifelse(SNP %in% hlight, "yes", "no")) %>% mutate( is_annotate=ifelse(P < threshold, "yes", "no"))

    axisdf <- df.tmp %>% group_by(CHR) %>% summarize(center=( max(BPcum) + min(BPcum) ) / 2 )

    plt = ggplot(df.tmp, aes(x=BPcum, y=-log10(P))) +
        # Show all points
        geom_point(aes(color=as.factor(CHR)), alpha=0.8, size=2) +
        scale_color_manual(values = rep(col, 22 ))+

        # custom X axis:
        scale_x_continuous( label = axisdf$CHR, breaks= axisdf$center ) +
        scale_y_continuous(expand = c(0, 0), limits = ylims) + # expand=c(0,0)removes space between plot area and x axis 

        # add plot and axis titles
        ggtitle(paste0(title)) +
        labs(x = "Chromosome") +     # add genome-wide sig and sugg lines
        geom_hline(yintercept = -log10(sig), color='firebrick') +
        geom_hline(yintercept = -log10(sugg), linetype="dashed", color='firebrick') + 

        geom_point(data=subset(df.tmp, is_highlight=="yes"), color="firebrick2", size=2) +
    
        # Add label using ggrepel to avoid overlapping
        geom_label_repel(data=df.tmp[df.tmp$is_annotate=="yes",], aes(label=as.factor(SNP), alpha=0.7), size=5, label.size=NA, force=1.3) +
    
    
    
        theme_bw(base_size = 22) +
        theme( 
          plot.title = element_text(hjust = 0.5),
          legend.position="none",
          panel.border = element_blank(),
          panel.grid.major.x = element_blank(),
          panel.grid.minor.x = element_blank())
    return(plt)
}

/dors/capra_lab/users/abraha1/conda/envs/py36_r_ml/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/dors/capra_lab/users/abraha1/conda/envs/py36_r_ml/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/dors/capra_lab/users/abraha1/conda/envs/py36_r_ml/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)


In [32]:
%%R -i output_dir -i uk_df  -h 500 -w 1200

highlight_snps= (uk_df %>% filter(P < 5e-8) %>% select(SNP))$SNP

threshold=5e-8
hlight=highlight_snps
col=c("#E2709A", "#CB4577")

ylims=c(0,-1*log10(min(uk_df$P))+2)
title="Replication GWAS"


png_file = file.path(output_dir, paste( Sys.Date(), "_ukbb_manplot.png", sep=""))


plt = gg.manhattan(uk_df, threshold, hlight, col, ylims, title) + theme_Publication(base_size=18) + theme(legend.position="none")

ggsave(png_file, plot=plt)



In [33]:
%%R -i bv_df  -h 500 -w 1200

highlight_snps= (bv_df %>% filter(P < 5e-8) %>% select(SNP))$SNP

threshold=5e-8
hlight=highlight_snps
col=c("#E2709A", "#CB4577")

ylims=c(0,-1*log10(min(bv_df$P))+2)
title="Discovery GWAS"


png_file = file.path(output_dir, paste( Sys.Date(), "_biovu_manplot.png", sep=""))


plt = gg.manhattan(bv_df, threshold, hlight, col, ylims, title) + theme_Publication(base_size=18) + theme(legend.position="none")
plt
# ggsave(pdf_file, plot=plt)
ggsave(png_file, plot=plt)